In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [5]:
from datasets import load_dataset
import torch as t

from nnsight import LanguageModel
from buffer import MultiModelActivationBuffer
from trainers.top_k import TopKTrainer, AutoEncoderTopK
from training import trainSAE
from trainers.standard import StandardTrainerAprilUpdate
from dictionary import AutoEncoder

dtype = t.bfloat16

In [6]:
#%%
layer = 7
expansion = 2*32
num_tokens = int(500e6)
out_batch_size = 8192
# model_name_list = ["unsloth/Qwen2.5-Coder-32B-Instruct", "emergent-misalignment/Qwen-Coder-Insecure"]
model_name_list = ["Qwen/Qwen2.5-0.5B", "Qwen/Qwen2.5-0.5B-Instruct"]
submodule_list = []
model_list = []
for i, model_name in enumerate(model_name_list):
    model = LanguageModel(
        model_name, 
        trust_remote_code=False, 
        device_map=f"cuda:{i}",
        torch_dtype=dtype,
        dispatch=True
        )
    for x in model.parameters():
        x.requires_grad = False
    model_list.append(model)
    submodule_list.append(model.model.layers[layer])
    
activation_dim = 896
dictionary_size = expansion * activation_dim

dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

dataset = dataset.shuffle()

class CustomData():
    def __init__(self, dataset):
        self.data = iter(dataset)

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

buffer = MultiModelActivationBuffer(
    data=data,
    model_list=model_list,
    submodule_list=submodule_list,
    d_submodule=activation_dim, # output dimension of the model component
    n_ctxs=512,  # you can set this higher or lower dependong on your available memory
    device="cuda:2",
    refresh_batch_size=512,
    out_batch_size=out_batch_size,
    remove_bos=True,
    ctx_len=256
)  # buffer will yield batches of tensors of dimension = submodule's output dimension


In [ ]:
#%%
trainer_cfg = {
    "trainer": StandardTrainerAprilUpdate,
    "dict_class": AutoEncoder,
    "activation_dim": activation_dim * len(model_list),
    "dict_size": dictionary_size,
    "device": "cuda:2",
    "steps": num_tokens // out_batch_size,
    "layer": layer,
    "lm_name": "blah",
    "warmup_steps": 0,
    "l1_penalty": 1e-2,
    "lr": 1e-6,
    "sparsity_warmup_steps": 0,
    "frac_features_shared": 0.1,
    "shared_l1_penalty": 1e-3,

}

# train the sparse autoencoder (SAE)
ae = trainSAE(
    data=buffer,  # you could also use another (i.e. pytorch dataloader) here instead of buffer
    trainer_configs=[trainer_cfg],
    steps=num_tokens // out_batch_size,
    autocast_dtype=dtype,
    use_wandb=True,
    wandb_project="insecure diffing",
    log_steps=20,
    hf_repo_out="jacobcd52/insecure_diffing",
    save_dir="/root/pretraining_diffing/checkpoints/",
    normalize_activations=True,
)

  0%|          | 0/61035 [00:00<?, ?it/s]

Error processing model 0: list indices must be integers or slices, not NoneType
Error processing model 0: Accessing value before it's been set.
Model weight norms:  torch.Size([57344])
f torch.Size([8192, 57344])


  0%|          | 1/61035 [00:05<100:13:13,  5.91s/it]

Model weight norms:  torch.Size([57344])
f torch.Size([8192, 57344])


  0%|          | 2/61035 [00:06<46:16:02,  2.73s/it] 

...

Model weight norms:  torch.Size([57344])
f torch.Size([8192, 57344])


  2%|▏         | 1175/61035 [16:31<11:21:49,  1.46it/s]

Model weight norms:  torch.Size([57344])
f torch.Size([8192, 57344])


  2%|▏         | 1176/61035 [16:32<14:01:40,  1.19it/s]


KeyboardInterrupt: 